In [1]:
#Import Statements
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time

In [2]:


#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
#from selenium import webdriver
#firefox_binary = FirefoxBinary()
#driver = webdriver.Firefox(firefox_binary=firefox_binary)

#driver = webdriver.Firefox()
#driver.get("http://www.python.org")
#assert "Python" in driver.title
#elem = driver.find_element(By.NAME, "q")
#elem.clear()
#elem.send_keys("pycon")
#elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
#driver.close()

In [3]:
#Load Rochelle Data
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")
#scraped_data_frame = pd.DataFrame() !!!LETS CREATE THE DATA FRAME 
#rochelle.head


In [4]:
#rochelle.columns

In [5]:
#rochelle.reset_index(inplace = True,drop = True) 
#name = rochelle.loc[2]["Scientific Name"] #for some reason couldn't access rochelle[0]...
#print(name)


In [6]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [7]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
# url = "https://quod.lib.umich.edu/h/herb00ic/x-1000138/MICH-V-1000138A?lasttype=boolean;lastview=reslist;op2=And;q2=Phragmites+australis+australis;resnum=1;rgn2=ic_all;select2=all;size=50;start=1;subview=detail;view=entry"
#html = urllib.request.urlopen(url, context=ctx).read()
#soup = BeautifulSoup(html, 'html.parser')
#soup

In [8]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
#resp = requests.get(url)
#newsoup = BeautifulSoup(resp.content, 'html.parser')
#iteminfo = soup.find('dl')
#print(iteminfo)

In [9]:
#accessing search page of the herbarium
herbarium_url = "https://quod.lib.umich.edu/cgi/i/image/image-idx?xc=1;page=searchgroup;g=herb-ic"
herbarium_html = urllib.request.urlopen(herbarium_url, context=ctx).read()
#herbarium_html


In [10]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?

#send driver to the herbarium website and submit a search query and scrape the amount of results
#BE SURE TO SWITCH ALL "FIND ELEMENTS" TO XPATH IF PLAUSIBLE?
driver.get(herbarium_url)
elem = driver.find_element(by = By.NAME, value = 'q1') #Q1 is the first search box?
elem.send_keys('Cabomba caroliniana') #Sends in a name from Rochelle's list
searchbutton = driver.find_element(by = By.CSS_SELECTOR, value= '#collection-search > button.\[.button.button--cta.\]') #selects the advanced search button based off the pulled tag from right clicking the inspected element
searchbutton.click()
#search_results_url = driver.current_url
resultcount = driver.find_element(by = By.CLASS_NAME, value ='results-heading').text
#next line returns a list of two numbers like ['23','23']
resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results
resultitem = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div[2]/section[1]/a')
resultitem.click()

#individually select each search result and scrape each matching entry

In [11]:
#driver2 = webdriver.Chrome()
scraped_dataframe = pd.DataFrame(index=[0])
for result in range(int(resultcountnumberlist[0]))[:3]: #[:-1]: #properly traverses the amount of results on the page
    scrapeddata1 = []
    scrapeddata2 = []
    time.sleep(2)
    resp = requests.get(driver.current_url)
    #print(resp.content)
    newsoup = BeautifulSoup(resp.content, 'html.parser')
    collection_category = newsoup.find_all('dt')
    collection_data = newsoup.find_all('dd')
    for tag in collection_category:
        scrapeddata1.append(tag.text)
    for tag in collection_data:
        scrapeddata2.append(tag.text)
    nextbutton = driver.find_element(by=By.XPATH, value='/html/body/main/div[1]/p/a[2]')
    nextbutton.click()
    scraped_data = dict(zip(scrapeddata1, scrapeddata2))
    result_df = pd.DataFrame(data=scraped_data, index=[0])
    #print(result_df)
    if scraped_dataframe.empty:
        scraped_dataframe = result_df
        #print('Empty')
    else:
        scraped_dataframe = scraped_dataframe.append(pd.DataFrame(data = result_df.values))
        #print('Not Empty Anymore')
    #print(result_df)
#scraped_dataframe.append(result_df)



In [12]:
scraped_dataframe

,0,1,2,3,4,5,6,7,8,9,...,Image Size,Institution Code,Link to this Item,Locality,Manifest,Modified,Occurrence ID,Record,Scientific Name,State/Province
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3753 x 5634,MICH,\n\nhttps://quod.lib.umich.edu/h/herb00ic/x-12...,"Methuen Twp., Bass Bay, Kasshabog Lake",\n\nhttps://quod.lib.umich.edu/cgi/i/image/api...,2016-3-11,df39a3bd-695e-4d5b-b21e-5586374c08f8,1201461,\nCabomba caroliniana\n,Ontario
0,44.6376900000,-77.9427600000,1201476,\nCabomba caroliniana\n,Cabombaceae,M.J. Oldham & D.A. Sutherland,Michael J. Oldham,22414,1999-07-12,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,42.7776300000,-83.5845300000,1302380,\nCabomba caroliniana\n,Cabombaceae,Anton A. Reznicek,10329,1996-09-04,North America,USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
scraped_dataframe.columns

Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,                  10,                  11,
                        12,                  13,                  14,
                        15,                  16,                  17,
                        18,                  19,                  20,
                        21,                  22,                  23,
                        24,                  25,                  26,
          'Catalog Number',        'Collection',   'Collection Code',
               'Collector',  'Collector Number',         'Continent',
                 'Country',            'County',  'Decimal Latitude',
       'Decimal Longitude',        'Event Date',            'Family',
               'File Size',     'Full citation',           'Habitat',
                    

In [14]:
aaa =result_df.columns

In [25]:
testdf = pd.DataFrame()
columns = []
for c in aaa:

    if c not in testdf.columns:
        #print(c)
        columns.append(c)
ntestdf = pd.DataFrame(columns=columns)

In [26]:
ntestdf.columns

Index(['Decimal Latitude', 'Decimal Longitude', 'Catalog Number',
       'Scientific Name', 'Family', 'Collector', 'Collector Number',
       'Event Date', 'Continent', 'Country', 'State/Province', 'County',
       'Locality', 'Habitat', 'Collection Code', 'Institution Code',
       'Modified', 'ID', 'Occurrence ID', 'Collection', 'Image Size',
       'File Size', 'Record', 'Link to this Item', 'Manifest',
       'Full citation'],
      dtype='object')

In [27]:
ntestdf

,Decimal Latitude,Decimal Longitude,Catalog Number,Scientific Name,Family,Collector,Collector Number,Event Date,Continent,Country,...,Modified,ID,Occurrence ID,Collection,Image Size,File Size,Record,Link to this Item,Manifest,Full citation
